<a href="https://colab.research.google.com/github/zjuer-dm/colab/blob/main/2d_gaussian_splatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before proceeding, please ensure that your notebook is connected to a GPU runtime.

In [1]:
!apt-get update
!apt-get install -y build-essential libglm-dev

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,050 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,561 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Pac

In [2]:
# 1. 克隆主仓库及其所有子模块
!echo ">>> 1. Cloning repository..."
!git clone --recursive https://github.com/hbb1/2d-gaussian-splatting.git
%cd 2d-gaussian-splatting

# 2. 安装第一个缺失的系统依赖 (用于解决第一个错误)
!echo ">>> 2. Installing libglm-dev..."
!apt-get update && apt-get install -y libglm-dev

# 3. 【关键修复】为 simple-knn 源码打补丁 (用于解决 FLT_MAX 错误)
!echo ">>> 3. Patching simple-knn source code..."
!sed -i '1i#include <cfloat>' /content/2d-gaussian-splatting/submodules/simple-knn/simple_knn.cu

# 4. 现在，执行完整的 pip 安装
!echo ">>> 4. Running pip install..."
!pip install ./submodules/diff-surfel-rasterization
!pip install ./submodules/simple-knn
!pip install -r requirements.txt

!echo ">>> Installation finished."
!pip install -q plyfile

>>> 1. Cloning repository...
Cloning into '2d-gaussian-splatting'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 274 (delta 130), reused 85 (delta 85), pack-reused 76 (from 2)
Receiving objects: 100% (274/274), 9.40 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Submodule 'submodules/diff-surfel-rasterization' (https://github.com/hbb1/diff-surfel-rasterization.git) registered for path 'submodules/diff-surfel-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/2d-gaussian-splatting/submodules/diff-surfel-rasterization'...
remote: Enumerating objects: 697, done.        
remote: Counting objects: 100% (285/285), done.        
remote: Compressing objects: 100% (29/29), done.        
remote: Total 697 (delta 269), reused 256 (delta 256), pack-reused 412 (f

In [3]:
%cd /content/2d-gaussian-splatting/

/content/2d-gaussian-splatting


In [4]:
import os
# os.environ['CUDA_HOME'] = '/usr/local/cuda' #ensure that the CUDA path is correct
# !pip install ninja
# !pip install submodules/simple-knn --verbose
# !pip install submodules/diff-surfel-rasterization --verbose

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/2d-gaussian-splatting/

/content/2d-gaussian-splatting


For Colab to access your COLMAP processed data, please copy the COLMAP output to your Google Drive and enter the full COLMAP source path (e.g. /content/drive/MyDrive/COLMAP_out)

In [7]:
COLMAP_PATH = str(input('Please enter COLMAP output path:'))

Please enter COLMAP output path:/content/drive/MyDrive/360_extra_scenes/flowers


In [8]:
!python train.py -s $COLMAP_PATH

2025-06-24 12:36:49.980320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750768610.000036    4786 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750768610.006049    4786 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-24 12:36:50.028199: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Optimizing 
Output folder: ./output/8f6e8467-4 [24/06 12:36:53]
Traceback (most recent call last):
  File "/content/2

Training is complete, to use render.py, please enter the desired model_path located in your Drive: /content/drive/MyDrive/2d_gaussian_splatting/output/{10 digit UUID}: (e.g: /content/drive/MyDrive/2d_gaussian_splatting/output/8d812792-1)

In [ ]:
MODEL_PATH = str(input('Please enter the model path:'))

In [ ]:
!pip3 install open3d==0.18.0 mediapy==1.1.2 lpips==0.1.4 scikit-image==0.21.0 tqdm==4.66.2 trimesh==4.3.2 opencv-python

In [ ]:
!python render.py -m $MODEL_PATH -s $COLMAP_PATH

Training and rendering is complete, please navigate to your drive and see the model outputs under the directory "2d_gaussian_splatting/".